In [1]:
!pip install langchain pinecone pypdf PyMuPDF huggingface_hub

In [1]:
!pip install python-dotenv

In [4]:
import os
from dotenv import load_dotenv
load_dotenv() 
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
pinecone_key = os.getenv("PINECONE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_TYPE"] = os.getenv("OPENAI_API_TYPE")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
drive_pdf_path="/content/drive/MyDrive/medical-chatbot/book.pdf"


In [5]:
import os
import shutil
os.makedirs("docs", exist_ok=True)
local_pdf_path = f"docs/{os.path.basename(drive_pdf_path)}"
shutil.copy(drive_pdf_path, local_pdf_path)

print(f"File copied to {local_pdf_path} and ready for processing")


File copied to docs/book.pdf and ready for processing


In [7]:
!pip install -U langchain-community
!pip install  langchain_community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [8]:
!pip install pymupdf
from langchain_community.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = DirectoryLoader("docs/", glob="*.pdf", loader_cls=PyMuPDFLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

print(f"Loaded {len(docs)} text chunks from your book")


Loaded 21771 text chunks from your book


In [9]:
import os
from pinecone import Pinecone, ServerlessSpec

# Create a Pinecone client
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

INDEX_NAME = "medicalchatbot"

if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=384,  # replace with your vector dimension
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

index = pc.Index(INDEX_NAME)

print("✅ Connected to Pinecone index:", INDEX_NAME)


✅ Connected to Pinecone index: medicalchatbot


In [16]:
!pip install langchain-pinecone


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 24.4 MB/s eta 0:00:00


In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = PineconeVectorStore.from_existing_index("medicalchatbot", embeddings)



In [19]:
retriever = vectorstore.as_retriever()

In [11]:
from langchain.llms import HuggingFaceEndpoint
from langchain.chains import RetrievalQA


In [12]:
!pip install --upgrade langchain openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.8/810.8 kB 55.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.100.0
    Uninstalling openai-1.100.0:
      Successfully uninstalled openai-1.100.0


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("OPENAI_API_BASE", "https://openrouter.ai/api/v1")
os.environ["OPENAI_API_TYPE"] = os.getenv("OPENAI_API_TYPE", "open_router")
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
print("✅ ChatOpenAI imported successfully")


/tmp/ipython-input-3206557745.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


✅ ChatOpenAI imported successfully


In [22]:
# Imports
from langchain.chains import RetrievalQA
from langchain.schema import Document


# 5. Define the retriever
retriever = vectorstore.as_retriever()


# Initialize the QA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# Ask a question
query = "What are the medicines of heartattack?"
result = qa({"query": query})

print("Answer:", result["result"])
for doc in result["source_documents"]:
    print("-", doc.page_content[:200], "...")


/tmp/ipython-input-2060553711.py:19: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": query})


Answer: Some of the medicines used for heart attacks include amiodarone, atropine, bretylium, disopyramide, lidocaine, procainamide, propafenone, propranolol, quinidine, and sotalol. Additionally, angiotensin-converting enzyme (ACE) inhibitors are also used to manage and prevent heart failure following a heart attack.
- the heart’s normal rhythm. These include: amiodarone
(Cordarone),
atropine,
bretylium,
disopyramide
(Norpace),
lidocaine
(Xylocaine),
procainamide
(Procan),
propafenone
(Rythmol),
propranolol
(Inderal ...
- the heart’s normal rhythm. These include: amiodarone
(Cordarone),
atropine,
bretylium,
disopyramide
(Norpace),
lidocaine
(Xylocaine),
procainamide
(Procan),
propafenone
(Rythmol),
propranolol
(Inderal ...
- 4:3190
for ventricular ectopic beats,
5:3911
See also Percutaneous translum-
inal coronary angioplasty
Angiosarcomas, liver biopsy for,
3:2236
Angiotensin, 4:2923
Angiotensin-converting enzyme
inhibit ...
- 4:3190
for ventricular ectopic beats,
5:3911
See also Per